<a href="https://colab.research.google.com/github/Deep-Learning-Challenge/challenge-notebooks/blob/master/1.Multilayer%20Perceptrons/3.Advanced%20Lessons/5.Lift%20Performance%20With%20Learning%20Rate%20Schedules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

# Lift Performance With Learning Rate Schedules

Training a neural network or a large deep learning model is a difficult optimization task. The classical algorithm to train neural networks is called stochastic gradient descent. It has been well established that you can achieve increased performance and faster training on some problems using a learning rate during training. In this lesson, you will discover how you can use different learning rate schedules for your neural network models in Python using the Keras deep learning library. After completing this lesson, you will know:

* The benefit of learning rate schedules on lifting model performance during training.
* How to configure and evaluate a time-based learning rate schedule.
* How to configure and evaluate a drop-based learning rate schedule.

Let's get started.

## Runtime Setup

In [1]:
import sys

dataset_name = "ionosphere.csv"
if 'google.colab' in sys.modules:
    DATASET = f"https://github.com/Deep-Learning-Challenge/challenge-notebooks/raw/master/datasets/{dataset_name}"
else:
    DATASET = f"../../datasets/{dataset_name}"
    
DATASET

'../../datasets/ionosphere.csv'

## Learning Rate Schedule For Training Models

Adapting the learning rate for your stochastic gradient descent optimization procedure can increase performance and reduce training time. Sometimes this is called learning rate annealing or adaptive learning rates. Here we will call this approach a learning rate schedule, where the default schedule is to use a constant learning rate to update network weights for each training epoch.

The most straightforward and perhaps most used adaptation of learning rates during training are techniques that reduce the learning rate over time. These benefit from making large changes at the beginning of the training procedure when larger learning rate values are used and decreasing the learning rate such that a lower rate and therefore smaller training updates are made to weights later in the training procedure. This has the effect of quickly learning good weights early and fine-tuning them later. Two popular and easy to use learning rate schedules are as follows:

* Decrease the learning rate gradually based on the epoch.
* Decrease the learning rate using punctuated large drops at specific epochs.

Next, we will look at how you can use each of these learning rate schedules in turn with Keras.

## Ionosphere Classification Dataset

The Ionosphere binary classification problem is used as a demonstration in this lesson. The dataset describes radar returns where the target was free electrons in the ionosphere. It is a binary classification problem where positive cases (g for good) show that some structures in the ionosphere and negative cases (b for bad) do not. It is a useful dataset for practicing with neural networks because all of the inputs are small numerical values of the same scale. There are 34 attributes and 351 observations.

State-of-the-art results on this dataset achieve an accuracy of approximately 94% to 98% accuracy using 10-fold cross-validation. You can learn more about the ionosphere dataset on the [UCI Machine Learning Repository
website](https://archive.ics.uci.edu/ml/datasets/Ionosphere).

## Time-Based Learning Rate Schedule

Keras has a time-based learning rate schedule built-in. The stochastic gradient descent optimization algorithm implementation in the SGD class has an argument called decay. This argument is used in the time-based learning rate decay schedule equation as follows:

$$ Learning Rate = LearningRate\times\frac{1}{1 + decay\times epoch}  $$

When the decay argument is zero (the default), this does not affect the learning rate (e.g., 0.1).

```
LearningRate = 0.1 * 1/(1 + 0.0 * 1)
LearningRate = 0.1
```

When the decay argument is specified, it will decrease the previous epoch's learning rate by the given fixed amount. For example, if we use the initial learning rate value of 0.1 and the decay of 0.001, the first five epochs will adapt the learning rate as follows:

```
Epoch Learning Rate
    1 0.1
    2 0.0999000999
    3 0.0997006985
    4 0.09940249103
    5 0.09900646517
```

Extending this out to 100 epochs will produce the following graph of learning rate (y-axis) versus epoch (x-axis):

![Learning Rate Decay](../../images/learning_rate_decay.png)

You can create a nice default schedule by setting the decay value as follows:

```
Decay = LearningRate / Epochs
Decay = 0.1 / 100
Decay = 0.001
```

The example below demonstrates using the time-based learning rate adaptation schedule in Keras. A small neural network model is constructed with a single hidden layer with 34 neurons and using the rectifier activation function. The output layer has a single neuron and uses the sigmoid activation function to output probability-like values. The learning rate for stochastic gradient descent has been set to a higher value of 0.1. The model is trained for 50 epochs, and the decay argument has been set to 0.002, calculated as $\frac{0.1}{50}$.
Additionally, it can be a good idea to use momentum when using an adaptive learning rate. In this case, we use a momentum value of 0.8. The complete example is listed below.

In [2]:
# Time Based Learning Rate Decay
from pandas import read_csv
import numpy

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = read_csv(DATASET, header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:34].astype(float)
Y = dataset[:,34]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

# create model
model = Sequential()
model.add(Dense(34, input_dim=34, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# Compile model
epochs = 50
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

2021-10-17 19:50:19.827987: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-17 19:50:19.828064: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-10-17 19:50:27.237481: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-17 19:50:27.237937: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-17 19:50:27.238435: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (33d7dde1ac92): /proc/driver/nvidia/version does not exist
2021-10-17 19:50:27.239968: I tensorflow/core/platform/cpu_featu

The model is trained on 67% of the dataset and evaluated using a 33% validation dataset. Running the example shows a classification accuracy of 99.14%. This is higher than the baseline of 95.69% without the learning rate decay or momentum.

In [3]:
# Fit the model
model.fit(X, Y, validation_split=0.33, epochs=epochs, batch_size=28, verbose=2)

2021-10-17 19:51:12.774731: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
9/9 - 2s - loss: 0.6853 - accuracy: 0.6085 - val_loss: 0.6653 - val_accuracy: 0.7586
Epoch 2/50
9/9 - 0s - loss: 0.6499 - accuracy: 0.7915 - val_loss: 0.6110 - val_accuracy: 0.7069
Epoch 3/50
9/9 - 0s - loss: 0.5719 - accuracy: 0.7830 - val_loss: 0.4954 - val_accuracy: 0.8879
Epoch 4/50
9/9 - 0s - loss: 0.4621 - accuracy: 0.8511 - val_loss: 0.3728 - val_accuracy: 0.9483
Epoch 5/50
9/9 - 0s - loss: 0.3708 - accuracy: 0.8723 - val_loss: 0.2651 - val_accuracy: 0.9741
Epoch 6/50
9/9 - 0s - loss: 0.3150 - accuracy: 0.8936 - val_loss: 0.2312 - val_accuracy: 0.9655
Epoch 7/50
9/9 - 0s - loss: 0.2701 - accuracy: 0.8894 - val_loss: 0.3756 - val_accuracy: 0.8448
Epoch 8/50
9/9 - 0s - loss: 0.2350 - accuracy: 0.9191 - val_loss: 0.1577 - val_accuracy: 0.9741
Epoch 9/50
9/9 - 0s - loss: 0.2311 - accuracy: 0.9106 - val_loss: 0.0950 - val_accuracy: 0.9741
Epoch 10/50
9/9 - 0s - loss: 0.2488 - accuracy: 0.9064 - val_loss: 0.1965 - val_accuracy: 0.9569
Epoch 11/50
9/9 - 0s - loss: 0.1829 - a

## Drop-Based Learning Rate Schedule

Another popular learning rate schedule used with deep learning models is to drop the learning rate at specific times during training systematically. Often this method is implemented by dropping the learning rate by half every fixed number of epochs. For example, we may have an initial learning rate of 0.1 and drop it by a factor of 0.5 every ten epochs. The fixed ten epochs of training would use a value of 0.1, in the next ten epochs, a learning rate of 0.05 would be used, and so on. If we plot out the learning rates for this example out to 100 epochs, you get the graph below-showing learning rate (y-axis) versus epoch (x-axis).

![Learning Rate Scheduke](../../images/learning_rate_schedule.png)

We can implement this in Keras using the `LearningRateScheduler` callback when fitting the model. The `LearningRateScheduler` callback allows us to define a function to call that takes the epoch number as an argument and returns the learning rate to use in stochastic gradient descent. When used, the learning rate specified by stochastic gradient descent is ignored. In the code below, we use the same example as a single hidden layer network on the Ionosphere dataset. A new `step_decay()` function is defined that implements the equation:

$$ LearningRate=InitialLearningRate \times {DropRate}^{floor(\frac{1+Epoch}{EpochDrop})} $$

`InitialLearningRate` is the learning rate at the beginning of the run, `EpochDrop` is how often the learning rate is dropped in epochs, and `DropRate` is how much to drop the learning rate each time it is dropped.

In [4]:
# Drop-Based Learning Rate Decay
from pandas import read_csv
import numpy
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler

from sklearn.preprocessing import LabelEncoder

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = read_csv(DATASET, header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:34].astype(float)
Y = dataset[:,34]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

# create model
model = Sequential()
model.add(Dense(34, input_dim=34, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# Compile model
sgd = SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

# learning schedule callback
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

Running the example results in a classification accuracy of 99.14% on the validation dataset, again an improvement over the baseline for the model on this dataset.

In [5]:
# Fit the model
model.fit(X, Y, validation_split=0.33, epochs=50, batch_size=28, callbacks=callbacks_list,
verbose=2)

Epoch 1/50
9/9 - 2s - loss: 0.6785 - accuracy: 0.6000 - val_loss: 0.5607 - val_accuracy: 0.9138
Epoch 2/50
9/9 - 0s - loss: 0.6141 - accuracy: 0.7149 - val_loss: 0.4691 - val_accuracy: 0.9397
Epoch 3/50
9/9 - 0s - loss: 0.4777 - accuracy: 0.8213 - val_loss: 0.3809 - val_accuracy: 0.9397
Epoch 4/50
9/9 - 0s - loss: 0.3608 - accuracy: 0.8809 - val_loss: 0.3406 - val_accuracy: 0.8707
Epoch 5/50
9/9 - 0s - loss: 0.3070 - accuracy: 0.8851 - val_loss: 0.4451 - val_accuracy: 0.7672
Epoch 6/50
9/9 - 0s - loss: 0.2442 - accuracy: 0.8979 - val_loss: 0.2163 - val_accuracy: 0.9224
Epoch 7/50
9/9 - 0s - loss: 0.2019 - accuracy: 0.9149 - val_loss: 0.1125 - val_accuracy: 0.9655
Epoch 8/50
9/9 - 0s - loss: 0.1821 - accuracy: 0.9404 - val_loss: 0.0933 - val_accuracy: 0.9741
Epoch 9/50
9/9 - 0s - loss: 0.1650 - accuracy: 0.9277 - val_loss: 0.1210 - val_accuracy: 0.9655
Epoch 10/50
9/9 - 0s - loss: 0.1405 - accuracy: 0.9532 - val_loss: 0.1322 - val_accuracy: 0.9741
Epoch 11/50
9/9 - 0s - loss: 0.1640 - a

## Tips for Using Learning Rate Schedules

This section lists some tips and tricks to consider when using learning rate schedules with neural networks.

* **Increase the initial learning rate**: Because the learning rate will decrease, start with a larger value to decrease from. A larger learning rate will result in much larger changes to the weights, at least in the beginning, allowing you to benefit from fine-tuning later.
* **Use a large momentum**: Using a larger momentum value will help the optimization algorithm continue to make updates in the right direction when your learning rate shrinks to small values.
* **Experiment with schedules**: It will not be clear which learning rate schedule to use, so try a few with different configuration options and see what works best on your problem. Also, try schedules that change exponentially and even schedules that respond to your model's accuracy on the training or test datasets.

## Summary

In this lesson, you discovered learning rate schedules for training neural network models. You learned:

* The benefits of using learning rate schedules during training to lift model performance.
* How to configure and use a time-based learning rate schedule in Keras.
* How to develop your drop-based learning rate schedule in Keras.